In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from keras.applications import  DenseNet121, VGG16, ResNet50, Xception, EfficientNetB7
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [1]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

tf.keras.backend.clear_session()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
dataset_dir = r'C:\Users\Metehan Sevgil\Desktop\KODLAR\Anaconda\Bitirme\lung_colon_image_set'

file_paths = []
labels = []

for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith('.jpeg'):
            file_paths.append(os.path.join(root, file))
            labels.append(os.path.basename(root))

data_df = pd.DataFrame({'filepaths': file_paths, 'labels': labels})
print(data_df.head())
print(data_df.tail())

                                           filepaths     labels
0  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  colon_aca
1  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  colon_aca
2  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  colon_aca
3  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  colon_aca
4  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  colon_aca
                                               filepaths    labels
24995  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  lung_scc
24996  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  lung_scc
24997  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  lung_scc
24998  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  lung_scc
24999  C:\Users\Metehan Sevgil\Desktop\KODLAR\Anacond...  lung_scc


In [9]:
max_epoch = 15
batch_size = 32
img_size = (128, 128)

tr_gen = ImageDataGenerator(rescale=1. / 255)
ts_gen = ImageDataGenerator(rescale=1. / 255)
vs_gen = ImageDataGenerator(rescale=1. / 255)

train_gen = tr_gen.flow_from_directory("./dataset/train",
                                        target_size=img_size, batch_size=batch_size, color_mode= 'rgb', class_mode='categorical')

test_gen = ts_gen.flow_from_directory("./dataset/test",
                                        target_size=img_size, batch_size=batch_size, color_mode= 'rgb', class_mode='categorical')

val_gen = vs_gen.flow_from_directory("./dataset/val",
                                        target_size=img_size, batch_size=batch_size, color_mode= 'rgb', class_mode='categorical')

Found 15000 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.


In [5]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

base_model.trainable = True

densenet121_128 = Sequential()
densenet121_128.add(base_model)

densenet121_128.add(Flatten())

densenet121_128.add(BatchNormalization())
densenet121_128.add(Dense(512, activation='relu'))
densenet121_128.add(Dropout(0.4))

densenet121_128.add(BatchNormalization())

densenet121_128.add(Dense(5, activation='softmax'))

densenet121_128.compile(optimizer=SGD(), 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

densenet121_128.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 4, 4, 1024)        7037504   
                                                                 
 flatten (Flatten)           (None, 16384)             0         
                                                                 
 batch_normalization (BatchN  (None, 16384)            65536     
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 512)               8389120   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                        

In [6]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [10]:
history_densenet121_128 = densenet121_128.fit(train_gen,
                                              validation_data=val_gen,
                                              epochs=max_epoch,
                                              callbacks=[early_stopping])

Epoch 1/15
469/469 [==============================] - 116s 237ms/step - loss: 0.1407 - accuracy: 0.9493 - val_loss: 0.0576 - val_accuracy: 0.9790
Epoch 2/15
469/469 [==============================] - 101s 216ms/step - loss: 0.0408 - accuracy: 0.9863 - val_loss: 0.0138 - val_accuracy: 0.9948
Epoch 3/15
469/469 [==============================] - 101s 216ms/step - loss: 0.0178 - accuracy: 0.9940 - val_loss: 0.0120 - val_accuracy: 0.9950
Epoch 4/15
469/469 [==============================] - 102s 218ms/step - loss: 0.0118 - accuracy: 0.9961 - val_loss: 0.0054 - val_accuracy: 0.9980
Epoch 5/15
469/469 [==============================] - 101s 215ms/step - loss: 0.0069 - accuracy: 0.9980 - val_loss: 0.0066 - val_accuracy: 0.9974
Epoch 6/15
469/469 [==============================] - 101s 215ms/step - loss: 0.0062 - accuracy: 0.9983 - val_loss: 0.0042 - val_accuracy: 0.9984
Epoch 7/15
469/469 [==============================] - 102s 218ms/step - loss: 0.0040 - accuracy: 0.9989 - val_loss: 0.0054 -

In [11]:
import json

with open("history_densenet121_128.json", "w") as f:
    json.dump(history_densenet121_128.history, f)

In [12]:
densenet121_128.save('densenet121_model_128.h5')